# Problema
Una determinada empresa forestal puede producir $L$ productos distintos y tiene $I$ plantas productivas ubicadas en diferentes zonas, siendo $S_i^t$ la capacidad total de producción de la planta $i, (i=1,…,l)$ en el período $t, (t=1,…,T)$ sin importar de qué tipo de producto se trate, $T$ el número total de periodos. El tipo de producto $l$ tiene un costo de producción de $P_l$ sin importar la planta que lo fabrique ni el período en cuestión. Los productos son demandados por $J$ ciudades diferentes, siendo $D_{ljt}$ la demanda de la ciudad $j, (j=1,…,J)$ por el producto $l, (l=1,…,L)$, en el período $t, (t=1,…,T)$. Las demandas deben satisfechas período a período.

Como no existe la posibilidad de almacenar producto en las plantas, la empresa está estudiando la posibilidad de arrendar bodegas ubicadas en diferentes puntos geográficos. El arriendo de las bodegas se hace período a período, esto quiere decir que, si se arrienda la bodega $k$ en el período $t$, no necesariamente la bodega $k$ debe haber estado arrendada el período $t-1$ o seguir arrendada para el período $t+1$. Hay $K$ posibles bodegas para arrendar. De esta manera, la producción de las plantas se llevará a las bodegas y desde allí se abastecerá a las ciudades. No existe inventario de productos, las bodegas sólo se utilizan para etiquetar los distintos artículos. Si se arrienda la bodega $k,(k=1,…,K)$ se incurre en un gasto fijo $F_{kt}$ pesos por el pago del arriendo en el período $t$. La capacidad de la bodega $k$ es de ${Q_k}$ unidades de producto sin importar su tipo.

El costo de transporte del producto l desde la planta i a la bodega $k$ en el período t es de $M_{likt}$ pesos y el costo de transporte desde la bodega $k$ a la ciudad $j$ del producto $l$ en el período $t$ es de $N_{lkjt}$ pesos.